### Tuesday, May 23rd, 2023

The cluster is down for maintenance this week, and I've been having reliability issues with Google Colab lately, so I'm using a GPU box I have at home for the time being.


In [1]:
import torch

import pyro
import pyro.distributions as dist

from torch_geometric.data import Data
from torch_geometric.datasets import KarateClub
from torch_geometric.utils import (
    stochastic_blockmodel_graph,
    to_networkx
)

import networkx as nx
import matplotlib.pyplot as plt

In [32]:
pyro.clear_param_store()
num_nodes = 127

n_blocks_logits = pyro.param("n_blocks_logits", torch.randn(num_nodes))
n_blocks_distrib = dist.Categorical(logits=n_blocks_logits)
n_blocks = pyro.sample("n_blocks", n_blocks_distrib)

block_assignments = []
for i in range(num_nodes):
    assignment_logits = pyro.param(f"assigment_logits_{i}", torch.randn(n_blocks))
    assignment_dist = dist.Categorical(logits=assignment_logits)
    
    block_assignments.append(
        pyro.sample(f"block_assignment_{i}", assignment_dist)
    )    
block_assignments = torch.Tensor(block_assignments).long()
block_assignments

#block_sizes = []
#for i in range(n_blocks):
#    block_sizes.append(
#        (block_assignments == i).sum()
#    )
#block_sizes = torch.Tensor(block_sizes).long()
#block_sizes

tensor([13, 12, 11,  9,  6, 10, 10, 13,  7, 11, 10,  2, 14,  1,  0, 15, 10,  5,
         8, 14, 15,  7,  4,  8, 14,  6, 15, 14,  2,  1,  5,  1,  6, 10,  9,  3,
        15,  7,  7,  1,  2, 15,  8,  5,  0,  9,  9,  8,  5,  9,  8, 15,  3,  5,
         1,  5, 14, 10,  2, 15,  9, 11,  1,  7, 13, 10,  0,  8,  6,  5,  4,  0,
         6,  7, 15, 11,  2,  3, 11, 14,  0, 11, 13,  3,  7, 11, 14,  6, 11, 10,
         8,  2, 11,  5,  9,  5,  2, 13, 11,  8, 15, 10,  7,  3,  5,  6,  6,  8,
         1,  6, 15,  0,  2,  2,  0,  8, 12,  1,  8,  7, 11, 12, 13, 11,  4,  1,
         9])

In [25]:
block_probs = pyro.param("block_probs", torch.rand(n_blocks, n_blocks))

In [27]:
edge_index = stochastic_blockmodel_graph(
    block_sizes,
    block_probs,
    directed=True
)

In [29]:
edge_index.shape

torch.Size([2, 7925])

In [45]:
torch.bernoulli(torch.tensor([[0.3, 0.4, 0.1, 1.0],[0.1, 0.3, 0.8, 0.2]]))

tensor([[0., 0., 0., 1.],
        [0., 1., 1., 0.]])

In [48]:
torch.combinations(torch.arange(num_nodes), r=2).t()

tensor([[  0,   0,   0,  ..., 124, 124, 125],
        [  1,   2,   3,  ..., 125, 126, 126]])

In [7]:
from torch_geometric.datasets import KarateClub

In [118]:
n_blocks = 4
min_size = 5
max_size = 10

block_sizes = torch.randint(min_size, max_size, size=(n_blocks,))
block_probs = torch.rand(n_blocks, n_blocks)
edge_index = stochastic_blockmodel_graph(
    block_sizes,
    block_probs,
    directed=True
)
#data = Data(x=torch.randn(block_sizes.sum()), edge_index=edge_index)
#G = to_networkx(data)

#plt.figure(figsize=(10,10), dpi=200)
#nx.draw_networkx(G, pos=nx.spectral_layout(G), with_labels=False, node_size=5, width=0.1, arrows=False)

In [119]:
edge_index.shape

torch.Size([2, 362])

In [120]:
kc = KarateClub().get(0)

In [121]:
kc.edge_index.shape

torch.Size([2, 156])